In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import sqldf
import gc
import missingno as msno

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
%%time
train_metadata = pd.read_csv('training_set_metadata.csv')
test_metadata = pd.read_csv('test_set_metadata.csv')

CPU times: user 4.15 s, sys: 374 ms, total: 4.53 s
Wall time: 4.5 s


In [4]:
%%time
sample_submission = pd.read_csv('sample_submission.csv')

CPU times: user 2.49 s, sys: 445 ms, total: 2.93 s
Wall time: 2.92 s


In [5]:
%%time
train_passband = pd.read_csv('agg_train_passband.csv')
test_passband = pd.read_csv('agg_test_passband.csv')
train_merged = pd.read_csv('agg_train_merged.csv')
test_merged = pd.read_csv('agg_test_merged.csv')

CPU times: user 1min 7s, sys: 4.16 s, total: 1min 11s
Wall time: 1min 11s


In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [7]:
%%time
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)
train_passband = reduce_mem_usage(train_passband)
test_passband = reduce_mem_usage(test_passband)
train_merged = reduce_mem_usage(train_merged)
test_merged = reduce_mem_usage(test_merged)

Memory usage of dataframe is 0.72 MB
Memory usage after optimization is: 0.18 MB
Decreased by 75.0%
Memory usage of dataframe is 293.14 MB
Memory usage after optimization is: 76.61 MB
Decreased by 73.9%
Memory usage of dataframe is 7.54 MB
Memory usage after optimization is: 2.83 MB
Decreased by 62.5%
Memory usage of dataframe is 3357.73 MB
Memory usage after optimization is: 1339.09 MB
Decreased by 60.1%
Memory usage of dataframe is 1.20 MB
Memory usage after optimization is: 0.45 MB
Decreased by 62.5%
Memory usage of dataframe is 532.97 MB
Memory usage after optimization is: 206.53 MB
Decreased by 61.2%
CPU times: user 14 s, sys: 11.3 s, total: 25.4 s
Wall time: 25.4 s


In [8]:
train_temp0 = train_passband[train_passband['passband'] == 0]
train_temp1 = train_passband[train_passband['passband'] == 1]
train_temp2 = train_passband[train_passband['passband'] == 2]
train_temp3 = train_passband[train_passband['passband'] == 3]
train_temp4 = train_passband[train_passband['passband'] == 4]
train_temp5 = train_passband[train_passband['passband'] == 5]

In [9]:
test_temp0 = test_passband[test_passband['passband'] == 0]
test_temp1 = test_passband[test_passband['passband'] == 1]
test_temp2 = test_passband[test_passband['passband'] == 2]
test_temp3 = test_passband[test_passband['passband'] == 3]
test_temp4 = test_passband[test_passband['passband'] == 4]
test_temp5 = test_passband[test_passband['passband'] == 5]

In [10]:
del train_passband
del test_passband
gc.collect()

7

# NO GROUPED LEFT JOIN

In [ ]:
train_merged.head()

In [ ]:
for i in list(train_merged):
    print("B.{} AS NG_{},".format(i,i))

In [ ]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS NG_count_flux_err,
B.min_flux_err AS NG_min_flux_err,
B.max_flux_err AS NG_max_flux_err,
B.std_flux_err AS NG_std_flux_err,
B.sum_flux_err AS NG_sum_flux_err,
B.mean_flux_err AS NG_mean_flux_err,
B.median_flux_err AS NG_median_flux_err,
B.count_flux AS NG_count_flux,
B.min_flux AS NG_min_flux,
B.max_flux AS NG_max_flux,
B.std_flux AS NG_std_flux,
B.sum_flux AS NG_sum_flux,
B.mean_flux AS NG_mean_flux,
B.median_flux AS NG_median_flux,
B.count_detected AS NG_count_detected,
B.std_detected AS NG_std_detected,
B.sum_detected AS NG_sum_detected,
B.mean_detected AS NG_mean_detected,
B.median_detected AS NG_median_detected FROM train_metadata A LEFT JOIN train_merged B ON A.object_id = B.object_id""")

In [ ]:
train_metadata.head()

In [ ]:
train_metadata.shape

In [ ]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS NG_count_flux_err,
B.min_flux_err AS NG_min_flux_err,
B.max_flux_err AS NG_max_flux_err,
B.std_flux_err AS NG_std_flux_err,
B.sum_flux_err AS NG_sum_flux_err,
B.mean_flux_err AS NG_mean_flux_err,
B.median_flux_err AS NG_median_flux_err,
B.count_flux AS NG_count_flux,
B.min_flux AS NG_min_flux,
B.max_flux AS NG_max_flux,
B.std_flux AS NG_std_flux,
B.sum_flux AS NG_sum_flux,
B.mean_flux AS NG_mean_flux,
B.median_flux AS NG_median_flux,
B.count_detected AS NG_count_detected,
B.std_detected AS NG_std_detected,
B.sum_detected AS NG_sum_detected,
B.mean_detected AS NG_mean_detected,
B.median_detected AS NG_median_detected FROM test_metadata A LEFT JOIN test_merged B ON A.object_id = B.object_id""")

In [ ]:
test_metadata.shape

In [ ]:
del train_merged
del test_merged
gc.collect()

In [ ]:
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)

# PASSBAND 0 

In [ ]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A0_count_flux_err,
B.min_flux_err AS A0_min_flux_err,
B.max_flux_err AS A0_max_flux_err,
B.std_flux_err AS A0_std_flux_err,
B.sum_flux_err AS A0_sum_flux_err,
B.mean_flux_err AS A0_mean_flux_err,
B.median_flux_err AS A0_median_flux_err,
B.count_flux AS A0_count_flux,
B.min_flux AS A0_min_flux,
B.max_flux AS A0_max_flux,
B.std_flux AS A0_std_flux,
B.sum_flux AS A0_sum_flux,
B.mean_flux AS A0_mean_flux,
B.median_flux AS A0_median_flux,
B.count_detected AS A0_count_detected,
B.std_detected AS A0_std_detected,
B.sum_detected AS A0_sum_detected,
B.mean_detected AS A0_mean_detected,
B.median_detected AS A0_median_detected FROM train_metadata A LEFT JOIN train_temp0 B ON A.object_id = B.object_id""")

In [ ]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A0_count_flux_err,
B.min_flux_err AS A0_min_flux_err,
B.max_flux_err AS A0_max_flux_err,
B.std_flux_err AS A0_std_flux_err,
B.sum_flux_err AS A0_sum_flux_err,
B.mean_flux_err AS A0_mean_flux_err,
B.median_flux_err AS A0_median_flux_err,
B.count_flux AS A0_count_flux,
B.min_flux AS A0_min_flux,
B.max_flux AS A0_max_flux,
B.std_flux AS A0_std_flux,
B.sum_flux AS A0_sum_flux,
B.mean_flux AS A0_mean_flux,
B.median_flux AS A0_median_flux,
B.count_detected AS A0_count_detected,
B.std_detected AS A0_std_detected,
B.sum_detected AS A0_sum_detected,
B.mean_detected AS A0_mean_detected,
B.median_detected AS A0_median_detected FROM test_metadata A LEFT JOIN test_temp0 B ON A.object_id = B.object_id""")

In [ ]:
del train_temp0,test_temp0
gc.collect()

In [ ]:
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)

# PASSBAND 1

In [ ]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A1_count_flux_err,
B.min_flux_err AS A1_min_flux_err,
B.max_flux_err AS A1_max_flux_err,
B.std_flux_err AS A1_std_flux_err,
B.sum_flux_err AS A1_sum_flux_err,
B.mean_flux_err AS A1_mean_flux_err,
B.median_flux_err AS A1_median_flux_err,
B.count_flux AS A1_count_flux,
B.min_flux AS A1_min_flux,
B.max_flux AS A1_max_flux,
B.std_flux AS A1_std_flux,
B.sum_flux AS A1_sum_flux,
B.mean_flux AS A1_mean_flux,
B.median_flux AS A1_median_flux,
B.count_detected AS A1_count_detected,
B.std_detected AS A1_std_detected,
B.sum_detected AS A1_sum_detected,
B.mean_detected AS A1_mean_detected,
B.median_detected AS A1_median_detected FROM train_metadata A LEFT JOIN train_temp1 B ON A.object_id = B.object_id""")

In [ ]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A1_count_flux_err,
B.min_flux_err AS A1_min_flux_err,
B.max_flux_err AS A1_max_flux_err,
B.std_flux_err AS A1_std_flux_err,
B.sum_flux_err AS A1_sum_flux_err,
B.mean_flux_err AS A1_mean_flux_err,
B.median_flux_err AS A1_median_flux_err,
B.count_flux AS A1_count_flux,
B.min_flux AS A1_min_flux,
B.max_flux AS A1_max_flux,
B.std_flux AS A1_std_flux,
B.sum_flux AS A1_sum_flux,
B.mean_flux AS A1_mean_flux,
B.median_flux AS A1_median_flux,
B.count_detected AS A1_count_detected,
B.std_detected AS A1_std_detected,
B.sum_detected AS A1_sum_detected,
B.mean_detected AS A1_mean_detected,
B.median_detected AS A1_median_detected FROM test_metadata A LEFT JOIN test_temp1 B ON A.object_id = B.object_id""")

In [ ]:
del train_temp1,test_temp1
gc.collect()

In [ ]:
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

In [ ]:
%%time
train_metadata.to_csv('train_metadata_NG_0_1.csv',index=False)
test_metadata.to_csv('test_metadata_NG_0_1.csv',index=False)

In [ ]:
gc.collect()

# PASSBAND 2

In [11]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A2_count_flux_err,
B.min_flux_err AS A2_min_flux_err,
B.max_flux_err AS A2_max_flux_err,
B.std_flux_err AS A2_std_flux_err,
B.sum_flux_err AS A2_sum_flux_err,
B.mean_flux_err AS A2_mean_flux_err,
B.median_flux_err AS A2_median_flux_err,
B.count_flux AS A2_count_flux,
B.min_flux AS A2_min_flux,
B.max_flux AS A2_max_flux,
B.std_flux AS A2_std_flux,
B.sum_flux AS A2_sum_flux,
B.mean_flux AS A2_mean_flux,
B.median_flux AS A2_median_flux,
B.count_detected AS A2_count_detected,
B.std_detected AS A2_std_detected,
B.sum_detected AS A2_sum_detected,
B.mean_detected AS A2_mean_detected,
B.median_detected AS A2_median_detected FROM train_metadata A LEFT JOIN train_temp2 B ON A.object_id = B.object_id""")

CPU times: user 410 ms, sys: 8.04 ms, total: 418 ms
Wall time: 417 ms


In [12]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A2_count_flux_err,
B.min_flux_err AS A2_min_flux_err,
B.max_flux_err AS A2_max_flux_err,
B.std_flux_err AS A2_std_flux_err,
B.sum_flux_err AS A2_sum_flux_err,
B.mean_flux_err AS A2_mean_flux_err,
B.median_flux_err AS A2_median_flux_err,
B.count_flux AS A2_count_flux,
B.min_flux AS A2_min_flux,
B.max_flux AS A2_max_flux,
B.std_flux AS A2_std_flux,
B.sum_flux AS A2_sum_flux,
B.mean_flux AS A2_mean_flux,
B.median_flux AS A2_median_flux,
B.count_detected AS A2_count_detected,
B.std_detected AS A2_std_detected,
B.sum_detected AS A2_sum_detected,
B.mean_detected AS A2_mean_detected,
B.median_detected AS A2_median_detected FROM test_metadata A LEFT JOIN test_temp2 B ON A.object_id = B.object_id""")

CPU times: user 2min 28s, sys: 5.78 s, total: 2min 33s
Wall time: 2min 33s


In [13]:
del train_temp2,test_temp2
gc.collect()

462

# PASSBAND 3

In [14]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A3_count_flux_err,
B.min_flux_err AS A3_min_flux_err,
B.max_flux_err AS A3_max_flux_err,
B.std_flux_err AS A3_std_flux_err,
B.sum_flux_err AS A3_sum_flux_err,
B.mean_flux_err AS A3_mean_flux_err,
B.median_flux_err AS A3_median_flux_err,
B.count_flux AS A3_count_flux,
B.min_flux AS A3_min_flux,
B.max_flux AS A3_max_flux,
B.std_flux AS A3_std_flux,
B.sum_flux AS A3_sum_flux,
B.mean_flux AS A3_mean_flux,
B.median_flux AS A3_median_flux,
B.count_detected AS A3_count_detected,
B.std_detected AS A3_std_detected,
B.sum_detected AS A3_sum_detected,
B.mean_detected AS A3_mean_detected,
B.median_detected AS A3_median_detected FROM train_metadata A LEFT JOIN train_temp3 B ON A.object_id = B.object_id""")

CPU times: user 477 ms, sys: 4.1 ms, total: 481 ms
Wall time: 480 ms


In [15]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A3_count_flux_err,
B.min_flux_err AS A3_min_flux_err,
B.max_flux_err AS A3_max_flux_err,
B.std_flux_err AS A3_std_flux_err,
B.sum_flux_err AS A3_sum_flux_err,
B.mean_flux_err AS A3_mean_flux_err,
B.median_flux_err AS A3_median_flux_err,
B.count_flux AS A3_count_flux,
B.min_flux AS A3_min_flux,
B.max_flux AS A3_max_flux,
B.std_flux AS A3_std_flux,
B.sum_flux AS A3_sum_flux,
B.mean_flux AS A3_mean_flux,
B.median_flux AS A3_median_flux,
B.count_detected AS A3_count_detected,
B.std_detected AS A3_std_detected,
B.sum_detected AS A3_sum_detected,
B.mean_detected AS A3_mean_detected,
B.median_detected AS A3_median_detected FROM test_metadata A LEFT JOIN test_temp3 B ON A.object_id = B.object_id""")

CPU times: user 3min 31s, sys: 6.49 s, total: 3min 37s
Wall time: 3min 37s


In [16]:
del train_temp3,test_temp3
gc.collect()

538

In [ ]:
%%time
train_metadata.to_csv('train_metadata_2_3.csv',index=False)
test_metadata.to_csv('test_metadata_2_3.csv',index=False)

# PASSBAND 4

In [11]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A4_count_flux_err,
B.min_flux_err AS A4_min_flux_err,
B.max_flux_err AS A4_max_flux_err,
B.std_flux_err AS A4_std_flux_err,
B.sum_flux_err AS A4_sum_flux_err,
B.mean_flux_err AS A4_mean_flux_err,
B.median_flux_err AS A4_median_flux_err,
B.count_flux AS A4_count_flux,
B.min_flux AS A4_min_flux,
B.max_flux AS A4_max_flux,
B.std_flux AS A4_std_flux,
B.sum_flux AS A4_sum_flux,
B.mean_flux AS A4_mean_flux,
B.median_flux AS A4_median_flux,
B.count_detected AS A4_count_detected,
B.std_detected AS A4_std_detected,
B.sum_detected AS A4_sum_detected,
B.mean_detected AS A4_mean_detected,
B.median_detected AS A4_median_detected FROM train_metadata A LEFT JOIN train_temp4 B ON A.object_id = B.object_id""")

CPU times: user 396 ms, sys: 33.5 ms, total: 430 ms
Wall time: 434 ms


In [12]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A4_count_flux_err,
B.min_flux_err AS A4_min_flux_err,
B.max_flux_err AS A4_max_flux_err,
B.std_flux_err AS A4_std_flux_err,
B.sum_flux_err AS A4_sum_flux_err,
B.mean_flux_err AS A4_mean_flux_err,
B.median_flux_err AS A4_median_flux_err,
B.count_flux AS A4_count_flux,
B.min_flux AS A4_min_flux,
B.max_flux AS A4_max_flux,
B.std_flux AS A4_std_flux,
B.sum_flux AS A4_sum_flux,
B.mean_flux AS A4_mean_flux,
B.median_flux AS A4_median_flux,
B.count_detected AS A4_count_detected,
B.std_detected AS A4_std_detected,
B.sum_detected AS A4_sum_detected,
B.mean_detected AS A4_mean_detected,
B.median_detected AS A4_median_detected FROM test_metadata A LEFT JOIN test_temp4 B ON A.object_id = B.object_id""")

CPU times: user 2min 19s, sys: 5.37 s, total: 2min 24s
Wall time: 2min 24s


In [13]:
del train_temp4,test_temp4
gc.collect()

352

# PASSBAND 5

In [14]:
%%time
train_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A5_count_flux_err,
B.min_flux_err AS A5_min_flux_err,
B.max_flux_err AS A5_max_flux_err,
B.std_flux_err AS A5_std_flux_err,
B.sum_flux_err AS A5_sum_flux_err,
B.mean_flux_err AS A5_mean_flux_err,
B.median_flux_err AS A5_median_flux_err,
B.count_flux AS A5_count_flux,
B.min_flux AS A5_min_flux,
B.max_flux AS A5_max_flux,
B.std_flux AS A5_std_flux,
B.sum_flux AS A5_sum_flux,
B.mean_flux AS A5_mean_flux,
B.median_flux AS A5_median_flux,
B.count_detected AS A5_count_detected,
B.std_detected AS A5_std_detected,
B.sum_detected AS A5_sum_detected,
B.mean_detected AS A5_mean_detected,
B.median_detected AS A5_median_detected FROM train_metadata A LEFT JOIN train_temp5 B ON A.object_id = B.object_id""")

CPU times: user 460 ms, sys: 3.91 ms, total: 464 ms
Wall time: 463 ms


In [15]:
%%time
test_metadata = sqldf("""SELECT A.*,B.count_flux_err AS A5_count_flux_err,
B.min_flux_err AS A5_min_flux_err,
B.max_flux_err AS A5_max_flux_err,
B.std_flux_err AS A5_std_flux_err,
B.sum_flux_err AS A5_sum_flux_err,
B.mean_flux_err AS A5_mean_flux_err,
B.median_flux_err AS A5_median_flux_err,
B.count_flux AS A5_count_flux,
B.min_flux AS A5_min_flux,
B.max_flux AS A5_max_flux,
B.std_flux AS A5_std_flux,
B.sum_flux AS A5_sum_flux,
B.mean_flux AS A5_mean_flux,
B.median_flux AS A5_median_flux,
B.count_detected AS A5_count_detected,
B.std_detected AS A5_std_detected,
B.sum_detected AS A5_sum_detected,
B.mean_detected AS A5_mean_detected,
B.median_detected AS A5_median_detected FROM test_metadata A LEFT JOIN test_temp5 B ON A.object_id = B.object_id""")

CPU times: user 3min 38s, sys: 6.55 s, total: 3min 45s
Wall time: 3min 45s


In [16]:
del train_temp5,test_temp5
gc.collect()

637

In [17]:
%%time
train_metadata.to_csv('train_metadata_4_5.csv',index=False)
test_metadata.to_csv('test_metadata_4_5.csv',index=False)

CPU times: user 3min 7s, sys: 1.71 s, total: 3min 8s
Wall time: 3min 9s
